In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [3]:
class Data():
    def __init__(self, path):
        self.path = path
    
    def extract(self):
        df = pd.read_csv(self.path)
        return df

In [12]:
data = Data('PMSM.csv')
df = data.extract()

In [ ]:
class Motor(nn.Module):
    def __init__(self):
        super(Motor, self).__init__()
        